In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

class BudgetWishlistML:
    def __init__(self):
        self.income = 0
        self.expenses = pd.DataFrame(columns=['Label', 'Amount'])
        self.wishlist = []
        self.model = None

    def check_budget_limits(self):
        budget_limit = self.income * 0.7
        if self.expenses['Amount'].sum() > budget_limit:
            print("Alert: Budget limit exceeded")

    def plot_expenses(self):
        if 'Month' not in self.expenses.columns:
            print("Month data is not available in expenses.")
            return

        monthly_expenses = self.expenses.groupby('Month').sum()
        plt.figure(figsize=(10, 6))
        plt.bar(monthly_expenses.index, monthly_expenses['Amount'])
        plt.xlabel('Month')
        plt.ylabel('Expenses')
        plt.title('Monthly Expenses')
        plt.show()

    def get_income(self):
        try:
            self.income = float(input("Enter your monthly income: "))
        except ValueError:
            print("Invalid input. Please enter a valid number.")
            self.get_income()

    def add_expense(self):
        label = input("Enter the label for the expense: ")
        try:
            amount = float(input(f"Enter the amount for {label}: "))
            month = input("Enter the month (e.g., January, February, etc.): ")
            self.expenses = self.expenses.append({'Label': label, 'Amount': amount, 'Month': month}, ignore_index=True)
        except ValueError:
            print("Invalid input. Please enter a valid number.")
            self.add_expense()

    def add_to_wishlist(self, item, price):
        self.wishlist.append((item, price))

    def train_model(self, savings_history, time_to_purchase_history):
        X = np.array(savings_history).reshape(-1, 1)
        y = np.array(time_to_purchase_history)

        self.model = LinearRegression()
        self.model.fit(X, y)

    def calculate_time_to_purchase_ml(self, savings_per_month):
        predicted_months_required = self.model.predict([[savings_per_month]])

        if predicted_months_required > 0:
            print(f"\nBased on machine learning, it will take approximately {predicted_months_required[0]:.2f} months to purchase your wishlist items.")
            self.savings_needed_per_item(predicted_months_required[0])
        else:
            print("\nCongratulations! You can purchase your wishlist items this month.")

    def savings_needed_per_item(self, total_months_required):
        print("\nSavings needed for each item:")
        for item, price in self.wishlist:
            savings_needed = price / total_months_required
            print(f"{item}: Save ${savings_needed:.2f} per month for {int(total_months_required)} months.")

    def suggest_expense_removal(self):

        self.expenses['TimeRequired'] = self.model.predict(self.expenses['Amount'].values.reshape(-1, 1))

        unnecessary_expenses = self.expenses[self.expenses['TimeRequired'] < 0]

        if not unnecessary_expenses.empty:
            print("\nSuggestions to improve savings:")
            print(unnecessary_expenses[['Label', 'Amount']])
            print("Consider removing these unnecessary expenses to save more.")

    def calculate_savings_plan(self, limit_type, limit_value):
        if limit_type == 'money':
            total_cost = sum(item[1] for item in self.wishlist)
            total_months_required = total_cost / limit_value
            self.savings_needed_per_item(total_months_required)
        elif limit_type == 'months':
            savings_per_month = sum(item[1] / limit_value for item in self.wishlist)
            self.calculate_time_to_purchase_ml(savings_per_month)

def run_budget_wishlist():
    budget_wishlist_ml = BudgetWishlistML()

    budget_wishlist_ml.get_income()

    num_expenses = int(input("Enter the number of monthly expenses: "))
    for _ in range(num_expenses):
        budget_wishlist_ml.add_expense()

    while True:
        item = input("Enter item for wishlist (or 'done' to finish): ").strip().lower()
        if item == 'done':
            break
        try:
            price = float(input(f"Enter the price of {item}: "))
        except ValueError:
            print("Invalid input. Please enter a valid number.")
            continue
        budget_wishlist_ml.add_to_wishlist(item, price)


    for _ in range(3):
        savings = budget_wishlist_ml.income - budget_wishlist_ml.expenses['Amount'].sum()
        budget_wishlist_ml.expenses = budget_wishlist_ml.expenses.drop(columns=['TimeRequired'], errors='ignore')
        total_cost = sum(item[1] for item in budget_wishlist_ml.wishlist)
        months_required = total_cost / savings
        budget_wishlist_ml.train_model([savings], [months_required])


    budget_wishlist_ml.calculate_time_to_purchase_ml(savings)


    budget_wishlist_ml.suggest_expense_removal()


    print("\nCalculate a savings plan:")
    limit_type = input("Enter 'money' if you want to set a money limit, or 'months' if you want to set a time limit: ").strip().lower()

    try:
        limit_value = float(input(f"Enter the {limit_type} limit: "))
    except ValueError:
        print("Invalid input. Please enter a valid number.")

    budget_wishlist_ml.calculate_savings_plan(limit_type, limit_value)

class ExpenseTracker:
    def __init__(self):
        self.budget_wishlist = BudgetWishlistML()

    def record_expense(self):
        self.budget_wishlist.add_expense()

    def view_expense_history(self):
        self.budget_wishlist.plot_expenses()

    def start(self):
        while True:
            print("\nExpense Tracking Menu:")
            print("1. Record an Expense")
            print("2. View Expense History")
            print("3. Back to Main Wishlist Menu")
            choice = input("Enter your choice (1/2/3): ")

            if choice == '1':
                self.record_expense()
            elif choice == '2':
                self.view_expense_history()
            elif choice == '3':
                break
            else:
                print("Invalid choice. Please enter a valid option.")

if __name__ == "__main__":
    while True:
        print("\nMain Wishlist Menu:")
        print("1. Manage Wishlist")
        print("2. Expense Tracking")
        print("3. Exit")
        main_choice = input("Enter your choice (1/2/3): ")

        if main_choice == '1':
            run_budget_wishlist()
        elif main_choice == '2':
            expense_tracker = ExpenseTracker()
            expense_tracker.start()
        elif main_choice == '3':
            break
        else:
            print("Invalid choice. Please enter a valid option.")



Main Wishlist Menu:
1. Manage Wishlist
2. Expense Tracking
3. Exit
Enter your choice (1/2/3): 1
Enter your monthly income: 123000
Enter the number of monthly expenses: 1
Enter the label for the expense: tax
Enter the amount for tax: 70000
Enter the month (e.g., January, February, etc.): January


<ipython-input-13-62bb259be565>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.expenses = self.expenses.append({'Label': label, 'Amount': amount, 'Month': month}, ignore_index=True)


Enter item for wishlist (or 'done' to finish): iphone
Enter the price of iphone: 4500000
Enter item for wishlist (or 'done' to finish): done

Based on machine learning, it will take approximately 84.91 months to purchase your wishlist items.

Savings needed for each item:
iphone: Save $53000.00 per month for 84 months.

Calculate a savings plan:
Enter 'money' if you want to set a money limit, or 'months' if you want to set a time limit: money
Enter the money limit: 4500

Savings needed for each item:
iphone: Save $4500.00 per month for 1000 months.

Main Wishlist Menu:
1. Manage Wishlist
2. Expense Tracking
3. Exit
Enter your choice (1/2/3): 2

Expense Tracking Menu:
1. Record an Expense
2. View Expense History
3. Back to Main Wishlist Menu
Enter your choice (1/2/3): 1
Enter the label for the expense: groceries
Enter the amount for groceries: 4500
Enter the month (e.g., January, February, etc.): February


<ipython-input-13-62bb259be565>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.expenses = self.expenses.append({'Label': label, 'Amount': amount, 'Month': month}, ignore_index=True)



Expense Tracking Menu:
1. Record an Expense
2. View Expense History
3. Back to Main Wishlist Menu
Enter your choice (1/2/3): 3

Main Wishlist Menu:
1. Manage Wishlist
2. Expense Tracking
3. Exit
Enter your choice (1/2/3): 3
